In [1]:
import pandas as pd

In [2]:
answer_class = pd.read_csv('data/answer_class.csv')

In [3]:
answer_class.head()

,answer_class,Answer
0,0,После успешного прохождения выпускных испытани...
1,1,"Чтобы получить итоговый документ, нужно сдать ..."
2,2,"Можем его выдать, если вы:\n\nоплатили обучени..."
3,3,"Можем его выдать, если вы:\n\nоплатили обучени..."
4,4,Диплом или удостоверение отправим бесплатно По...


In [4]:
train_data = pd.read_csv('data/train_data.csv')

In [5]:
train_data.head()

,Question,Category,answer_class
0,"Что делать, если я хочу изменить группу или сп...",Перевод/ запись в группу,11
1,Из чего состоит основное обучение в личном каб...,Портал,15
2,Что входит в программу помощи с трудоустройств...,Трудоустройство,27
3,"Подскажите расписание каникул в 2024, пожалуйста",Расписание,21
4,Какие инструменты и ресурсы вы предоставляете ...,Трудоустройство,25


In [6]:
import catboost

In [7]:
from models.embeddings import Embeddings

In [8]:
import torch
device = torch.device('cuda:0')

In [9]:
embeddings = Embeddings('intfloat/multilingual-e5-large', device)

In [10]:
questions = list(train_data['Question'])

In [11]:
emb = embeddings.exec(questions)

In [12]:
emb

tensor([[-0.2244, -0.4083, -0.8827,  ..., -1.0480, -0.2838,  0.5377],
        [ 0.1340, -0.4035, -1.1452,  ..., -0.7542, -0.4170,  0.0190],
        [-0.3134, -0.6869, -1.0049,  ..., -0.8384, -0.9151,  0.5894],
        ...,
        [ 0.2009, -0.7030, -0.5857,  ..., -0.8074, -0.9111, -0.3514],
        [-0.4202, -0.4763, -0.0432,  ..., -0.7897, -1.2396,  0.0376],
        [ 0.1370, -0.8453, -1.7399,  ..., -1.1135, -0.8155,  0.2783]])

In [13]:
y = list(train_data['Category'])

In [14]:
X = emb.numpy()

In [15]:
model = catboost.CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, loss_function='MultiClass')

model.fit(X, y, verbose=100)

0:	learn: 2.2573662	total: 175ms	remaining: 17.3s
99:	learn: 0.2920936	total: 12.6s	remaining: 0us


In [16]:
aug_data = pd.read_csv('data/aug_train_data.csv')

In [21]:
aug_data = aug_data.drop('Unnamed: 0', axis=1)

In [44]:
def proc(k: int):
    row = aug_data.iloc[k]
    a = model.predict(embeddings.exec([row['Question']]).numpy())[0][0]
    b = row['Category']
    return a, b

In [47]:
from sklearn.metrics import f1_score

a = list(aug_data['Category'])
b = [proc(i)[0] for i in range(len(aug_data))]

In [49]:
f1_score(a, b, average='micro')

0.927536231884058

In [51]:
torch.cuda.empty_cache()